In [ ]:
# IMPORTS
import random
import numpy as np
import funcionesAuxiliares as fun_aux
import funcionAjuste
import funcionesVisualizacion as fun_vis
import funcionesTest
import matplotlib.pyplot as plt
! pip install joblib
from joblib import Parallel, delayed
! pip install tellurium
import tellurium as te
plt.ion
%matplotlib inline

hrs = np.arange(0,8760,0.1)
print('OK')


# LUZ AM AUTOMATIZADA (v. 9 puntos)

In [ ]:
#-----------------------------------------------------------------------------
# LOCALIZACION -> Indicar localizacion (ficheros adjuntos)
Localizacion = 'Albacete'
#-----------------------------------------------------------------------------
# REFERENCIA -> Indicar si se van a usar ficheros de referencia
Referencia = True
# Referencia = False
Latitud = 0
Longitud = 0 
PuntosReferencia = [] # Vector para introducir manualmente los puntos de rs
#-----------------------------------------------------------------------------
# MINIMOS Y MAXIMOS DE CADA ALEATORIO
# Diario (Para aumentar o disminuir radiación base)
MinimoDiario = 0.8
MaximoDiario = 1.15
# Horario (Para simular nubes)
MinimoHorario = 0.75
MaximoHorario = 1
#-----------------------------------------------------------------------------
# AJUSTE -> Activar para quitar hueco en ubicaciones del norte (ajustar horas de dia y noche)
Ajuste = True
epsilonValues = np.arange(-1,1,0.05) # Modificar para el ajuste
# deltaValues = np.arange(-1,1,0.05) # Modificar para el ajuste (NO SE PUEDE PASAR A LA FUNCION QUE PARARELIZA EL BUCLE -> SE DECLARA EN funcionesTest)
#-----------------------------------------------------------------------------
# TEST -> Activar para comprobar la duracion del dia y la noche de cada dia
Test = True
DebugMode = True
simulaRADIACION = False
#-----------------------------------------------------------------------------

Lectura de datos necesarios para el calculo de la radiación solar mediante los ficheros de referencia

In [ ]:
resul_leeReferencia = fun_aux.leeReferencia(Localizacion)
RReferencia = resul_leeReferencia[0]
Latitud = resul_leeReferencia[1]
Longitud =resul_leeReferencia[2]
x1 = resul_leeReferencia[3] # Puntos en los que se toman los datos de referencia

Calculo del fotoperiodo y las variables y1, y2 e y3

In [ ]:
resul_calculaFotoperiodo = fun_aux.calcula_y1(Latitud,x1,DebugMode)
y1 = resul_calculaFotoperiodo[0]
Fotoperiodo = resul_calculaFotoperiodo[1]
print(y1)

In [ ]:
hsol = funcionAjuste.ajuste(y1,x1) # el ajuste se ha cambiado por una interpolacion cubica (parece que los resultados salen mucho mejor)
plt.plot(x1,y1,'*',hrs,hsol)
print(len(hsol))

In [ ]:
resul_calculay23 = fun_aux.calcula_y23(RReferencia,y1,DebugMode)
y2 = resul_calculay23[0]
y3 = resul_calculay23[1]

Calculo de la radiación final (sin ajuste)

In [ ]:
Ac2 = funcionAjuste.ajuste(y2,x1)
plt.plot(x1,y2,'*',hrs,Ac2)
s = fun_aux.calculaRadiacion(Ac2, hsol)
fun_vis.muestraRadiacion(x1,y3,s)

In [ ]:
plt.xlim(172*24,173*24)
plt.plot(hrs,s)

### RESULTADOS ANTES DEL AJUSTE

In [ ]:
# Antes del ajuste con un margen de error del 30% en 1 hora
funcionesTest.ejecutaTest(Test,DebugMode,Latitud,s)

In [ ]:
def ajusteDeltaEpsilon(epsilon):
    mejorSuma = 1000000
    deltaValues=np.arange(-1,1,0.05)
    for delta in deltaValues:
            duracion = 0
            duracionInvierno = 0

            sAjuste = funcionesTest.radAjustada(y1,y2,x1,delta,epsilon)

            for i in sAjuste[43800:44040]: 
                if i == 0:
                    duracion = duracion + 1
            duracion = duracion/10
            diferenciaAUX = abs((24 - max(Fotoperiodo))-duracion)
            
            for i in sAjuste[0:240]:
                if i == 0:
                    duracionInvierno = duracionInvierno + 1
            duracionInvierno = duracionInvierno/10
            diferenciaAUXinvierno = abs((24 - min(Fotoperiodo))-duracionInvierno)
            sumaAUX = diferenciaAUXinvierno + diferenciaAUX
            
            if sumaAUX < mejorSuma:
                mejorSuma = sumaAUX
                mejorEpsilon = epsilon
                mejorDelta = delta
                if DebugMode:
                    print('MEJOR SUMA = ', mejorSuma, '-->',mejorEpsilon,'|',mejorDelta,'\n')
                
    return mejorSuma,mejorEpsilon, mejorDelta

In [ ]:
if Ajuste:
    results = Parallel(n_jobs=-1, verbose=1, backend="threading")(map(delayed(ajusteDeltaEpsilon), epsilonValues))
    minimo = 100000
    for i in range(len(results)):
        if results[i][0] < minimo:
            minimo = results[i][0] 
            print('MEJOR SUMA = ', results[i][0], '-->',results[i][1],'|',results[i][2],'\n')
            mejorepsilon = results[i][1]
            mejordelta = results[i][2]
else:
    print('Test desactivado')

### RESULTADOS DESPUES DEL AJUSTE

In [ ]:
if Ajuste:
    s = funcionesTest.radAjustada(y1,y2,x1,mejordelta,mejorepsilon)
    plt.plot(hrs,s)
    funcionesTest.ejecutaTest(Test,DebugMode,Latitud,s)

### RESULTADOS CON ALEATORIOS

In [ ]:
sFinal = []
for dia in range(0,365,1):
    randomDiario = random.uniform(MinimoDiario,MaximoDiario)
    for hora in range(0,24,1):
        randomHorario = random.uniform(MinimoHorario,MaximoHorario)
        for i in range(0,10,1):
            #print((hora*10+i)+240*dia)
            sFinal.append(s[(hora*10+i)+240*dia]*1000*randomDiario*randomHorario)
fun_vis.muestraRadiacion(x1,y3,sFinal)

In [ ]:
print(len(sFinal))
fun_vis.muestraInvierno(x1,y3,sFinal,7)
fun_vis.muestraVerano(x1,y3,sFinal,7)

In [ ]:
fsalida = open('radiacion_eventos.txt', 'w')
cont = 0
for i in range(87600):
    if sFinal[i] != 0:
        fsalida.write('at (time >= %f && time < %f): F1AUX = %f\n'%(i/10,(i+1)/10,sFinal[i]))
        ultimo = i-1
    if i == 0:
        i_inicial = 0
    if i != 0 and i!= 87599:
        if sFinal[i] == 0 and sFinal[i-1] != 0:
            i_inicial = i

        if sFinal[i] == 0 and sFinal[i+1] != 0:
            i_final = i
            fsalida.write('at (time >= %f && time < %f): F1AUX = 0.0\n'%(i_inicial/10,i_final/10))
               
fsalida.write('end\n')
fsalida.close()
print('TERMINADO')

In [ ]:
filenames = ['modeloFinal.txt','radiacion_eventos.txt'] 
with open('modeloFinal_v2.txt', 'w') as outfile: 
    for fname in filenames: 
        with open(fname) as infile: 
            for line in infile: 
                outfile.write(line)

Carga del modelo

In [ ]:

import time
tic = time.perf_counter()
print('Cargando fichero...')
model = te.loada('modeloFinal_v2.txt')
toc = time.perf_counter()
print(f'Fichero cargado en {toc - tic:0.2f} segundos')

Simulación del modelo

In [ ]:
n = 8760
print('Realizando simulaciones, espere por favor...')
tic = time.perf_counter()
model.resetAll()
simulation_result_especies = model.simulate(0.1,n,10*n+1)
print('1/2')
toc = time.perf_counter()
print(f'Simulacion ejecutada en {toc - tic:0.2f} segundos')

In [ ]:
if simulaRADIACION:
    n = 8760
    print('Realizando simulaciones, espere por favor...')
    tic = time.perf_counter()
    model.resetAll()
    simulation_result_F1 = model.simulate(0.1,n,10*n+1,['time','F1_ORIGINAL'])
    print('2/2')
    toc = time.perf_counter()
    print(f'Simulacion ejecutada en {toc - tic:0.2f} segundos')
else:
    print('Desactivado')

In [ ]:
#print(simulation_result_especies)

In [ ]:
fsalida = open('resultados.txt', 'w')
str = repr(simulation_result_especies)
fsalida.write(str)
fsalida.close()
if simulaRADIACION:
    fsalida = open('resultadosF1.txt', 'w')
    str = repr(simulation_result_F1)
    fsalida.write(str)
    fsalida.close()

In [ ]:
fichero = "resultados.txt"
with open(fichero,'r', encoding='utf-8') as f:
    data = f.read()
    data = data.replace("[", "")
    data = data.replace("]", "")
    data = data.replace(" ", "")
    with open(fichero,'w', encoding='utf-8') as f2:
        f2.write(data)
if simulaRADIACION:
    fichero = "resultadosF1.txt"
    with open(fichero,'r', encoding='utf-8') as f:
        data = f.read()
        data = data.replace("[", "")
        data = data.replace("]", "")
        data = data.replace(" ", "")
        with open(fichero,'w', encoding='utf-8') as f2:
            f2.write(data)